# Introduction

Notebook para fazer análise de modelos e de resultados de experimentos feitos e armazenados no Neptune

# Import Libraries

In [2]:
import os
import sys
import neptune
import zipfile
import shutil
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

if '../..' not in sys.path:
    sys.path.insert(0, '../..')

import config

# Download de Modelos

## Fetch Project

In [3]:
icao_nets_proj = neptune.init('guilhermemg/icao-nets-training', api_token=config.NEPTUNE_API_TOKEN)

## Fetch Experiments

In [4]:
df = icao_nets_proj.get_leaderboard(owner='guilhermemg')

In [5]:
df.to_csv('single_task_exps_data/experiments_data.csv', index=False)

## Select Experiments

In [6]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    train_val_test = train_val_test[3:]
    return train_val_test

In [7]:
df = pd.read_csv('single_task_exps_data/experiments_data.csv')
df = df[['id','property_icao_reqs','property_aligned','property_gt_names', 'created']]
df = df.dropna()
df = df[df.property_icao_reqs.str.contains('\[')]
df['property_gt_names'] = df.property_gt_names.apply(lambda row : process_gt_names(row))

init_date_exps = datetime(2021, 5, 25)
final_date_exps = datetime(2021, 5, 27)

df['created'] = df['created'].astype('datetime64[ns]')

df = df[(df.created >=  init_date_exps) & (df.created <  final_date_exps)]

df = df[(df.property_gt_names == 'fvc') & (df.property_aligned == False)]

display(df.head(5))
print(df.shape)

assert df.shape[0] == 23 # 23 experiments (FVC, NOT_ALIGNED) !!!

,id,property_icao_reqs,property_aligned,property_gt_names,created
186,IC-540,['mouth'],False,fvc,2021-05-25 02:59:05.091
187,IC-541,['rotation'],False,fvc,2021-05-25 04:11:22.507
188,IC-542,['l_away'],False,fvc,2021-05-25 05:12:09.412
189,IC-543,['eyes_closed'],False,fvc,2021-05-25 06:12:53.979
190,IC-545,['hat'],False,fvc,2021-05-25 08:21:44.326


(23, 5)


### Save Data to New .csv

In [8]:
df.to_csv('single_task_exps_data/processed_experiments_data.csv', index=False)

## Download Models

In [9]:
if '../../../../../notebooks/' not in sys.path:
    sys.path.insert(0, '../../../../../notebooks/')

from utils import constants as cts

In [10]:
def __check_dest_folder(dest_folder):
    if os.path.exists(dest_folder):
        shutil.rmtree(dest_folder)


def download_model(req, ds, align):
    print(f' Requisite: {req.upper()} | Dataset: {ds.upper()} | Aligned: {align}')
    
    req_name = f"['{req}']"
    
    exp_id = df[(df.property_icao_reqs.str.contains(req_name, regex=False, case=False)) & 
                 (df.property_aligned == align) &
                (df.property_gt_names.str.contains(ds))].id.values[0]
    print(f'.. Identified experiment id: {exp_id}')

    my_exp = icao_nets_proj.get_experiments(id=exp_id)[0]
    print('.. Downloading artifacts...')
    aligned = 'aligned' if align else 'not_aligned'
    dest_dir = f'prev_trained_models/single_task/{ds}_{aligned}/{req}'
    
    __check_dest_folder(dest_dir)
    
    print(f' .. Destination Folder: {dest_dir}')
    my_exp.download_artifacts(destination_dir=dest_dir)
    print(' .. Download done!')
    
    with zipfile.ZipFile(os.path.join(dest_dir, 'output.zip'), 'r') as zip_ref:
        zip_ref.extractall(dest_dir)
    
    os.remove(os.path.join(dest_dir, 'output.zip'))
    shutil.move(os.path.join(dest_dir, 'output', 'variables'), dest_dir)
    shutil.move(os.path.join(dest_dir, 'output', 'saved_model.pb'), dest_dir)
    shutil.rmtree(os.path.join(dest_dir, 'output'))
    
    print('.. Folders set')
    print('-----------------------------')

In [11]:
reqs = cts.ICAO_REQ.list_reqs_names()
datasets = ['fvc']
aligns = [False]

for req in reqs:
    for ds in datasets:
        for alig in aligns:
            download_model(req, ds, alig)

 Requisite: MOUTH | Dataset: FVC | Aligned: False
.. Identified experiment id: IC-540
.. Downloading artifacts...
 .. Destination Folder: prev_trained_models/single_task/fvc_not_aligned/mouth
 .. Download done!
.. Folders set
-----------------------------
 Requisite: ROTATION | Dataset: FVC | Aligned: False
.. Identified experiment id: IC-541
.. Downloading artifacts...
 .. Destination Folder: prev_trained_models/single_task/fvc_not_aligned/rotation
 .. Download done!
.. Folders set
-----------------------------
 Requisite: L_AWAY | Dataset: FVC | Aligned: False
.. Identified experiment id: IC-542
.. Downloading artifacts...
 .. Destination Folder: prev_trained_models/single_task/fvc_not_aligned/l_away
 .. Download done!
.. Folders set
-----------------------------
 Requisite: EYES_CLOSED | Dataset: FVC | Aligned: False
.. Identified experiment id: IC-543
.. Downloading artifacts...
 .. Destination Folder: prev_trained_models/single_task/fvc_not_aligned/eyes_closed
 .. Download done!
.